# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



## Word2Vec

The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [3]:
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


And here I'm creating dictionaries to convert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [5]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [6]:
from collections import Counter

word_freq = dict(Counter(int_words))

total_count = len(int_words)

subprobs = {w: 1 - np.sqrt(1e-5/(v /total_count)) for w, v in word_freq.items()}

## Your code here
train_words = [w for w in int_words if np.random.rand() > subprobs[w]]

len(train_words) / len(int_words)

0.27734957239844926

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [7]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
    R = np.random.randint(1, 1+window_size)
    start = idx - R if idx - R > 0 else 0
    end = idx + R + 1
    return set(words[start:idx] +  words[(idx+1):end])

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [8]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## Building the graph

From [Chris McCormick's blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as integers. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [9]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

## Embedding



The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use `tf.nn.embedding_lookup` to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform).

In [10]:
n_vocab = len(int_to_vocab)
n_embedding =  200 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform([n_vocab, n_embedding], -1, 1))# create embedding weight matrix here
    embed = tf.nn.embedding_lookup(embedding, inputs)# use tf.nn.embedding_lookup to get the hidden layer output

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [11]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal([n_vocab, n_embedding], stddev=0.01))# create softmax weight matrix here
    softmax_b = tf.Variable(tf.zeros(n_vocab))# create softmax biases here
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, 
                                      softmax_b,
                                      labels, 
                                      embed,
                                      n_sampled,
                                      n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [12]:
import random

In [13]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
# If the checkpoints directory doesn't exist:
#!mkdir checkpoints

## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [ ]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 5.4108 0.0207 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 5.3437 0.0172 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 5.2641 0.0179 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 5.3569 0.0178 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 5.2836 0.0177 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 5.2456 0.0176 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 5.1816 0.0172 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 5.1003 0.0172 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 4.9648 0.0178 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 4.8764 0.0175 sec/batch
Nearest to than: hooker, london, christians, threat, contains, edward, rejected, renewed,
Nearest to their: option, highly, does, hopping, available, chaplin, blew, deposited,
Nearest to more: fastest, administration, puts, barca, questions, justice, influx, quick,
Nearest to four: lose, faster, wh

Epoch 1/10 Iteration: 4100 Avg. Training loss: 4.5288 0.0181 sec/batch
Epoch 1/10 Iteration: 4200 Avg. Training loss: 4.5326 0.0175 sec/batch
Epoch 1/10 Iteration: 4300 Avg. Training loss: 4.4528 0.0175 sec/batch
Epoch 1/10 Iteration: 4400 Avg. Training loss: 4.4647 0.0175 sec/batch
Epoch 1/10 Iteration: 4500 Avg. Training loss: 4.4764 0.0175 sec/batch
Epoch 1/10 Iteration: 4600 Avg. Training loss: 4.4900 0.0171 sec/batch
Epoch 2/10 Iteration: 4700 Avg. Training loss: 4.4388 0.0132 sec/batch
Epoch 2/10 Iteration: 4800 Avg. Training loss: 4.4228 0.0171 sec/batch
Epoch 2/10 Iteration: 4900 Avg. Training loss: 4.3577 0.0173 sec/batch
Epoch 2/10 Iteration: 5000 Avg. Training loss: 4.3533 0.0173 sec/batch
Nearest to than: noblesse, lilac, cryptologia, journeyman, arabesque, barcode, operated, hooker,
Nearest to their: deposited, navel, fearless, pasha, option, sacrifices, supplication, hopping,
Nearest to more: needlework, puts, lash, barca, hemophilia, regressive, tefillin, insecticides,
N

Epoch 2/10 Iteration: 8100 Avg. Training loss: 4.2631 0.0186 sec/batch
Epoch 2/10 Iteration: 8200 Avg. Training loss: 4.1938 0.0179 sec/batch
Epoch 2/10 Iteration: 8300 Avg. Training loss: 4.3107 0.0181 sec/batch
Epoch 2/10 Iteration: 8400 Avg. Training loss: 4.2963 0.0179 sec/batch
Epoch 2/10 Iteration: 8500 Avg. Training loss: 4.3307 0.0175 sec/batch
Epoch 2/10 Iteration: 8600 Avg. Training loss: 4.2113 0.0187 sec/batch
Epoch 2/10 Iteration: 8700 Avg. Training loss: 4.2212 0.0175 sec/batch
Epoch 2/10 Iteration: 8800 Avg. Training loss: 4.3028 0.0177 sec/batch
Epoch 2/10 Iteration: 8900 Avg. Training loss: 4.1570 0.0174 sec/batch
Epoch 2/10 Iteration: 9000 Avg. Training loss: 4.2058 0.0174 sec/batch
Nearest to than: noblesse, tcm, forty, operated, psychomotor, allende, renault, piercer,
Nearest to their: navel, pasha, deposited, fearless, sacrifices, option, glorified, casas,
Nearest to more: needlework, truso, tefillin, hemophilia, puts, imr, heals, flattery,
Nearest to four: chee, s

Epoch 3/10 Iteration: 12100 Avg. Training loss: 4.1716 0.0188 sec/batch
Epoch 3/10 Iteration: 12200 Avg. Training loss: 4.1715 0.0185 sec/batch
Epoch 3/10 Iteration: 12300 Avg. Training loss: 4.1467 0.0180 sec/batch
Epoch 3/10 Iteration: 12400 Avg. Training loss: 4.1510 0.0179 sec/batch
Epoch 3/10 Iteration: 12500 Avg. Training loss: 4.1257 0.0174 sec/batch
Epoch 3/10 Iteration: 12600 Avg. Training loss: 4.0836 0.0184 sec/batch
Epoch 3/10 Iteration: 12700 Avg. Training loss: 4.1454 0.0188 sec/batch
Epoch 3/10 Iteration: 12800 Avg. Training loss: 4.0948 0.0181 sec/batch
Epoch 3/10 Iteration: 12900 Avg. Training loss: 4.1498 0.0182 sec/batch
Epoch 3/10 Iteration: 13000 Avg. Training loss: 4.1881 0.0181 sec/batch
Nearest to than: noblesse, edward, sophisticated, tcm, forty, abounds, renault, beauregard,
Nearest to their: navel, pasha, deposited, casas, geode, preference, glorified, fearless,
Nearest to more: needlework, truso, influx, benefit, flattery, tefillin, waqf, puts,
Nearest to fo

Epoch 4/10 Iteration: 16100 Avg. Training loss: 4.0234 0.0180 sec/batch
Epoch 4/10 Iteration: 16200 Avg. Training loss: 4.0706 0.0172 sec/batch
Epoch 4/10 Iteration: 16300 Avg. Training loss: 4.0885 0.0171 sec/batch
Epoch 4/10 Iteration: 16400 Avg. Training loss: 4.0237 0.0171 sec/batch
Epoch 4/10 Iteration: 16500 Avg. Training loss: 4.0470 0.0170 sec/batch
Epoch 4/10 Iteration: 16600 Avg. Training loss: 4.0364 0.0169 sec/batch
Epoch 4/10 Iteration: 16700 Avg. Training loss: 4.0800 0.0172 sec/batch
Epoch 4/10 Iteration: 16800 Avg. Training loss: 4.0598 0.0175 sec/batch
Epoch 4/10 Iteration: 16900 Avg. Training loss: 4.0791 0.0173 sec/batch
Epoch 4/10 Iteration: 17000 Avg. Training loss: 4.0538 0.0171 sec/batch
Nearest to than: noblesse, sophisticated, edward, keitai, forty, abounds, renault, tcm,
Nearest to their: pasha, navel, casas, deposited, preference, geode, glorified, highly,
Nearest to more: truso, needlework, influx, benefit, tefillin, lesser, flattery, puts,
Nearest to four: 

...

Epoch 7/10 Iteration: 28100 Avg. Training loss: 3.8926 0.0182 sec/batch
Epoch 7/10 Iteration: 28200 Avg. Training loss: 3.8888 0.0173 sec/batch
Epoch 7/10 Iteration: 28300 Avg. Training loss: 3.9023 0.0171 sec/batch
Epoch 7/10 Iteration: 28400 Avg. Training loss: 3.8979 0.0166 sec/batch
Epoch 7/10 Iteration: 28500 Avg. Training loss: 3.8568 0.0171 sec/batch
Epoch 7/10 Iteration: 28600 Avg. Training loss: 3.9459 0.0172 sec/batch
Epoch 7/10 Iteration: 28700 Avg. Training loss: 3.9298 0.0173 sec/batch
Epoch 7/10 Iteration: 28800 Avg. Training loss: 3.9379 0.0171 sec/batch
Epoch 7/10 Iteration: 28900 Avg. Training loss: 3.8684 0.0172 sec/batch
Epoch 7/10 Iteration: 29000 Avg. Training loss: 3.8699 0.0171 sec/batch
Nearest to than: foreshadowed, sophisticated, keitai, noblesse, forty, devote, abounds, edward,
Nearest to their: highly, to, pasha, preference, obliged, casas, dissociate, geode,
Nearest to more: truso, benefit, influx, lesser, mestizos, needlework, fastest, indifferent,
Nearest

Epoch 7/10 Iteration: 32100 Avg. Training loss: 3.9406 0.0187 sec/batch
Epoch 7/10 Iteration: 32200 Avg. Training loss: 3.9167 0.0179 sec/batch
Epoch 7/10 Iteration: 32300 Avg. Training loss: 3.9394 0.0173 sec/batch
Epoch 8/10 Iteration: 32400 Avg. Training loss: 3.9541 0.0034 sec/batch
Epoch 8/10 Iteration: 32500 Avg. Training loss: 3.9028 0.0175 sec/batch
Epoch 8/10 Iteration: 32600 Avg. Training loss: 3.8767 0.0173 sec/batch
Epoch 8/10 Iteration: 32700 Avg. Training loss: 3.8874 0.0174 sec/batch
Epoch 8/10 Iteration: 32800 Avg. Training loss: 3.9080 0.0173 sec/batch
Epoch 8/10 Iteration: 32900 Avg. Training loss: 3.8283 0.0172 sec/batch
Epoch 8/10 Iteration: 33000 Avg. Training loss: 3.8830 0.0175 sec/batch
Nearest to than: foreshadowed, keitai, sophisticated, noblesse, forty, devote, abounds, according,
Nearest to their: to, highly, obliged, preference, casas, amoeboid, dissociate, geode,
Nearest to more: truso, benefit, indifferent, eaux, mestizos, needlework, lesser, apologists,


Epoch 8/10 Iteration: 36100 Avg. Training loss: 3.9722 0.0184 sec/batch
Epoch 8/10 Iteration: 36200 Avg. Training loss: 3.9810 0.0173 sec/batch
Epoch 8/10 Iteration: 36300 Avg. Training loss: 3.9006 0.0175 sec/batch
Epoch 8/10 Iteration: 36400 Avg. Training loss: 3.8657 0.0170 sec/batch
Epoch 8/10 Iteration: 36500 Avg. Training loss: 3.8874 0.0172 sec/batch
Epoch 8/10 Iteration: 36600 Avg. Training loss: 3.8386 0.0173 sec/batch
Epoch 8/10 Iteration: 36700 Avg. Training loss: 3.8665 0.0176 sec/batch
Epoch 8/10 Iteration: 36800 Avg. Training loss: 3.8902 0.0175 sec/batch
Epoch 8/10 Iteration: 36900 Avg. Training loss: 3.8738 0.0175 sec/batch
Epoch 8/10 Iteration: 37000 Avg. Training loss: 3.9163 0.0173 sec/batch
Nearest to than: foreshadowed, forty, sophisticated, keitai, noblesse, devote, abounds, beg,
Nearest to their: to, obliged, amoeboid, preference, highly, casas, they, pasha,
Nearest to more: benefit, truso, eaux, secularization, indifferent, programmability, apologists, compensat

Epoch 9/10 Iteration: 40100 Avg. Training loss: 3.8785 0.0183 sec/batch
Epoch 9/10 Iteration: 40200 Avg. Training loss: 3.8560 0.0173 sec/batch
Epoch 9/10 Iteration: 40300 Avg. Training loss: 3.8416 0.0174 sec/batch
Epoch 9/10 Iteration: 40400 Avg. Training loss: 3.8547 0.0170 sec/batch
Epoch 9/10 Iteration: 40500 Avg. Training loss: 3.9267 0.0171 sec/batch
Epoch 9/10 Iteration: 40600 Avg. Training loss: 3.8553 0.0173 sec/batch
Epoch 9/10 Iteration: 40700 Avg. Training loss: 3.9797 0.0173 sec/batch
Epoch 9/10 Iteration: 40800 Avg. Training loss: 3.9124 0.0172 sec/batch
Epoch 9/10 Iteration: 40900 Avg. Training loss: 3.9414 0.0176 sec/batch
Epoch 9/10 Iteration: 41000 Avg. Training loss: 3.8383 0.0171 sec/batch
Nearest to than: forty, foreshadowed, keitai, devote, sophisticated, noblesse, according, most,
Nearest to their: to, they, amoeboid, raiders, obliged, highly, preference, wilf,
Nearest to more: benefit, truso, indifferent, eaux, apologists, various, secularization, most,
Nearest

Epoch 10/10 Iteration: 44100 Avg. Training loss: 3.8686 0.0185 sec/batch
Epoch 10/10 Iteration: 44200 Avg. Training loss: 3.8734 0.0174 sec/batch
Epoch 10/10 Iteration: 44300 Avg. Training loss: 3.8598 0.0169 sec/batch
Epoch 10/10 Iteration: 44400 Avg. Training loss: 3.8494 0.0173 sec/batch
Epoch 10/10 Iteration: 44500 Avg. Training loss: 3.8728 0.0170 sec/batch
Epoch 10/10 Iteration: 44600 Avg. Training loss: 3.8591 0.0173 sec/batch
Epoch 10/10 Iteration: 44700 Avg. Training loss: 3.8525 0.0174 sec/batch
Epoch 10/10 Iteration: 44800 Avg. Training loss: 3.8895 0.0173 sec/batch
Epoch 10/10 Iteration: 44900 Avg. Training loss: 3.8235 0.0172 sec/batch


Restore the trained network if you need to:

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)